# Day 3: Mull It Over

"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

---

## The Problem

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like `mul(X,Y)`, where `X` and `Y` are each 1-3 digit numbers. For instance:

- `mul(44,46)` multiplies 44 by 46 to get a result of 2024.
- Similarly, `mul(123,4)` would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a `mul` instruction. Sequences like:

- `mul(4*`
- `mul(6,9!`
- `?(12,34)`
- `mul ( 2 , 4 )`

do nothing.

---

## Example

Consider the following section of corrupted memory:

```
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
```

Only the four highlighted sections are real `mul` instructions:

- `mul(2,4)` → 2 * 4 = 8
- `mul(5,5)` → 5 * 5 = 25
- `mul(11,8)` → 11 * 8 = 88
- `mul(8,5)` → 8 * 5 = 40

Adding up the results of each instruction produces:

```
8 + 25 + 88 + 40 = 161
```

---

## Task

Scan the corrupted memory for uncorrupted `mul` instructions. **What do you get if you add up all of the results of the multiplications?**

In [1]:
from typing import Callable, Tuple
import re


def mul(a: int, b: int) -> int:
    return a * b


def get_muls(instructions: str) -> list[Tuple[str, str]]:
    # valid substrings are mul(1-3digit,1-3digit)
    # using capturing groups to extract the 1-3 digit numbers
    return re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", instructions)


def sum_valid_memory_muls(
    memory_file: str,
    get_muls_func: Callable[[str], list[Tuple[str, str]]],
    mul_func: Callable[[int, int], int],
) -> int:
    with open(memory_file, "r", encoding="utf-8") as f:
        instructions = f.read()
    muls = get_muls_func(instructions)
    return sum([mul_func(int(a), int(b)) for a, b in muls])


print(f"{sum_valid_memory_muls('example_1.txt', get_muls, mul)=}")
print(f"{sum_valid_memory_muls('input.txt', get_muls, mul)=}")

sum_valid_memory_muls('example_1.txt', get_muls, mul)=161
sum_valid_memory_muls('input.txt', get_muls, mul)=174336360


# Part Two

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

---

## New Instructions

There are two new instructions you'll need to handle:

1. `do()` enables future `mul` instructions.
2. `don't()` disables future `mul` instructions.

Only the most recent `do()` or `don't()` instruction applies. At the beginning of the program, `mul` instructions are **enabled**.

---

## Example

Consider the following corrupted memory:

```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
```

This corrupted memory is similar to the example from before, but this time:

- `mul(5,5)` and `mul(11,8)` are **disabled** because there is a `don't()` instruction before them.
- The other `mul` instructions function normally, including the one at the end, which gets **re-enabled** by a `do()` instruction.

---

### Calculation

- `mul(2,4)` → Enabled → 2 * 4 = 8
- `mul(5,5)` → Disabled
- `mul(32,64)` → Disabled
- `mul(11,8)` → Disabled
- `mul(8,5)` → Enabled (due to the `do()` instruction) → 8 * 5 = 40

Adding up the results of just the enabled `mul` instructions produces:

```
8 + 40 = 48
```

---

## Task

Handle the new instructions; **what do you get if you add up all of the results of just the enabled multiplications?**

In [2]:
def filter_instructions(instructions: str) -> str:
    # remove everything between a "don't()" and a "do()" (inclusive)
    # catch any in between characters and newlines!!
    return re.sub(r"(don't\(\)(.|\n)*?do\(\))", "", instructions)

print(f"{sum_valid_memory_muls('example_2.txt', lambda x: get_muls(filter_instructions(x)), mul)=}")
print(f"{sum_valid_memory_muls('input.txt', lambda x: get_muls(filter_instructions(x)), mul)=}")

sum_valid_memory_muls('example_2.txt', lambda x: get_muls(filter_instructions(x)), mul)=48
sum_valid_memory_muls('input.txt', lambda x: get_muls(filter_instructions(x)), mul)=88802350
